In [76]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
pd.set_option('display.max_colwidth', 100)

In [2]:
path = '../data/casaData/'

In [3]:
df_matchedListings = pd.read_csv(path + 'trainMatchedListings.csv')
df_matchedListings = df_matchedListings.reset_index()
df_matchedListings.columns = ['id1', 'id2','match']
# orig_id = set(df_all['id_clean'].tolist())
# df_matchedListings_clean = df_matchedListings.loc[(df_matchedListings['id2'].isin(orig_id)) & (df_matchedListings['id1'].isin(orig_id))]

In [4]:
df_matchedListings.head()

,id1,id2,match
0,1,2,1
1,1,3,1
2,1,4,1
3,1,5,1
4,1,6,1


In [5]:
df_matchedListings['id_list'] = list(zip(df_matchedListings['id1'], df_matchedListings['id2']))
df_matchedListings['id_list'] = df_matchedListings['id_list'].apply(sorted)
df_matchedListings['id_list'] = df_matchedListings['id_list'].apply(lambda x: ''.join(str(n) for n in x))
df_matchedListings = df_matchedListings.drop_duplicates(subset='id_list')

In [6]:
df_matchedListings.head()

,id1,id2,match,id_list
0,1,2,1,12
1,1,3,1,13
2,1,4,1,14
3,1,5,1,15
4,1,6,1,16


In [7]:
df_matchedListings.shape

(1817869, 4)

In [8]:
df_matchedListings = df_matchedListings.drop_duplicates('id1').reset_index(drop=True)

In [9]:
df_matchedListings.shape

(64953, 4)

In [10]:
id_list=[]
id_todrop=[]
for index, row in df_matchedListings.iterrows():
      if row['match']==1:
            min_id = row[['id1','id2']].min()
            id_list.append(min_id)
            id_todrop.append(row['id2'])
      else:
            id_list.append(row['id1'])

In [11]:
print(id_list[-10:-1])
print(id_todrop[-10:-1])

[104998, 105000, 105014, 105020, 105021, 105031, 105032, 105041, 105043]
[105025, 105027, 105029, 105030, 105031, 105034, 105039, 105040, 105043]


In [12]:
df_uniqueListings = df_matchedListings[~df_matchedListings['id1'].isin(id_todrop)]

In [13]:
df_uniqueListings = df_uniqueListings.reset_index(drop=True)

In [14]:
df_uniqueListings = df_uniqueListings[['id1']]

In [160]:
# df_uniqueListings.to_csv('unique_id.csv')

Merging with listings and cleaning text

In [15]:
df_unique_id = pd.read_csv('unique_id.csv', index_col=0)

In [16]:
df = pd.read_csv(path + 'trainListings.csv', header=0, usecols=[x for x in range(17)])

/var/folders/fm/m7sbfrtn37dcz2lvxhvj6q0m0000gn/T/ipykernel_29670/2889492285.py:1: DtypeWarning: Columns (0,4,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + 'trainListings.csv', header=0, usecols=[x for x in range(17)])


In [17]:
df_num_clean = pd.read_csv('trainListings_num.csv', index_col=0)

In [18]:
df_num_clean = df_num_clean[['id_clean', 'groupName', 'price_filled', 'bedroom_cats', 'bathroom_cats', 'area_totalLiving_filled']]

In [19]:
# turn id to numeric and remove ones that are not numeric 
df['id_clean'] = pd.to_numeric(df['id'], errors='coerce')
# removes non-numbers that became nans after to_numeric
df = df.dropna(subset='id_clean')
# convert to int
df['id_clean'] = df['id_clean'].astype(int)

In [20]:
df_num_clean.head()

,id_clean,groupName,price_filled,bedroom_cats,bathroom_cats,area_totalLiving_filled
0,1,house,26500000.0,6+,6+,4000.0
1,2,house,26500000.0,6+,6+,3000.0
2,3,house,26500000.0,6+,6+,3000.0
3,4,house,26500000.0,6+,6+,3000.0
4,5,house,26500000.0,6+,6+,517.0


In [21]:
df_num_clean.columns

Index(['id_clean', 'groupName', 'price_filled', 'bedroom_cats',
       'bathroom_cats', 'area_totalLiving_filled'],
      dtype='object')

In [22]:
df_all = pd.merge(df_num_clean, df, how='left', left_on='id_clean', right_on='id_clean')

In [23]:
df_all = df_all[['id_clean', 'groupName', 'price_filled', 'bedroom_cats',
       'bathroom_cats', 'area_totalLiving_filled', 'title', 'description', 'features']]

In [24]:
df_unique = pd.merge(df_unique_id, df_all, how='left', left_on='id1', right_on='id_clean')

In [25]:
df_unique = df_unique.drop('id1', axis=1)

In [26]:
df_unique = df_unique.dropna(subset='id_clean')
df_unique = df_unique.dropna(subset='title')
df_unique = df_unique.dropna(subset='description')

In [75]:
# df_unique['price'] = df_unique['price_filled'].astype(str).map('{:,}'.format)

In [156]:
# # drop rows that have missing both description and features
# df_unique = df_unique[~((df_unique['description'].isna()) & (df_unique['features'].isna()))]

In [28]:
df_unique['features'] = df_unique['features'].fillna('')

In [29]:
df_unique['all_text'] = df_unique['description'] + ' ' + df_unique['features']

In [40]:
# drop rows that have missing title description and features
# df_unique_text = df_unique_text[~(df_unique_text['title'].isna() & df_unique_text['description'].isna() & df_unique_text['features'].isna())]

In [30]:
from modules.preprocess_text import preprocess_text

In [31]:
df_unique['clean_text'] = preprocess_text(df_unique['all_text'], alpha=True)

In [35]:
def process_title(text_list):
    text_clean=[]
    for text in text_list:
        text = re.sub(r'https?://\S+|www\.\S+', '', text) # remove urls
        text = re.sub(r'<.*?>', '', text) # remove htmls
        text = re.sub(r'\s+', ' ', text).strip() #removes trailing white spaces and changes multiple spaces to one
        text = re.sub(r'(?<=[/.,])(?=[^\s])', ' ', text) # adds a space after a period or comma, avoid joining two words
        text = re.sub(r'[-/]', '', text)
        text = re.sub(r'\w*\d\w*', '', text)
        
        word_list = text.split()
        text=" ".join([word for word in word_list if not word.isnumeric()])

        word_list = text.split()
        text=" ".join([word for word in word_list if len(word)>1])

        text = text.title()
        text_clean.append(text)
    return text_clean

def process_description(text_list):
    text_clean=[]
    for text in text_list:
        text = re.sub(r'https?://\S+|www\.\S+', '', text) # remove urls
        text = re.sub(r'<.*?>', '', text) # remove htmls
        text = re.sub(r'\s+', ' ', text).strip() #removes trailing white spaces and changes multiple spaces to one
        text = re.sub(r'(?<=[/.,])(?=[^\s])', ' ', text) # adds a space after a period or comma, avoid joining two words
        # text = ' '.join(re.split(r'(?<=[.?!])\s+', text, 5)[:-1])
        text_clean.append(text)
    return text_clean

In [36]:
df_unique['clean_title'] = process_title(df_unique['title'])
df_unique['clean_description'] = process_description(df_unique['description'])

In [38]:
df_unique = df_unique[~df_unique['clean_description'].isna()]

In [39]:
from gensim.models import Word2Vec
import nltk
import spacy

In [40]:
nlp = spacy.load('en_core_web_lg')

In [29]:
# train nlp on text data - word embeddings were outputed as a csv 
with nlp.disable_pipes(): # disable pipe because already cleaned
    listing_desc = [nlp(x).vector for x in df_unique['clean_text']]

In [41]:
# importing word embeddings from the corpus
desc_vects = pd.read_csv('description_vectors.csv', index_col=0)

In [48]:
def recommender(text_description, num_rec=3):
    # turn to list for preprocessing
    # text_description = [text_description]
    # clean text 
    text_clean = preprocess_text([text_description], alpha=True)
    
    # get vector for input
    with nlp.disable_pipes():
        temp_desc = [nlp(text_clean[0]).vector]
    
    # get cosine similarities from all listings
    result = cosine_similarity(temp_desc, desc_vects)

    # argsort to get top 5 indices
    top5 = result.argsort()[0][-num_rec:]

    top5_df = pd.DataFrame()
    top5_titles =[]
    top5_prices=[]
    top5_desc=[]
    top5_type=[]
    for idx in top5:
        top5_titles.append(df_unique.iloc[idx]['clean_title'])
        top5_type.append(df_unique.iloc[idx]['groupName'])
        top5_prices.append(df_unique.iloc[idx]['price_filled'])
        top5_desc.append(df_unique.iloc[idx]['clean_description'])
    
    top5_df['Listing Name'] = top5_titles
    top5_df['Description'] = top5_desc
    top5_df['Price'] = top5_prices
    top5_df['Type'] = top5_type
    
    return top5_df

In [45]:
pd.set_option('display.max_colwidth', None)

In [74]:
recommender('apartment that is walking distance from a beach with a terrace')

,Listing Name,Description,Price,Type
0,Spacious Apartment Walking Distance To The Beach,"Spacious apartment in perfect condition with large terrace facing south, 2 minutes walk from the beach in Palmanova, in a quiet area near the beach promenade, near restaurants, shops and public transport. The apartment has a living room with access to a large terrace with views, fitchen kitchen, 3 bedrooms and 2 bathrooms. In addition there are communal pool and parking. MallorcaSite, your real estate agency in Mallorca, recommends this property for being in a quiet area, being in perfect condition, being near the beach, having a large terrace.",280000.0,apartment
1,Apartment In Cala Vinyas,"Sea view, ideally located apartment. 2 bedrooms with 2 ensuite bathrooms. Walking distance to the beach at Cala Vinyas with spacious terrrace, community pool and private parking. Walking distance to shops, bars and restaurants in Cala Vinyas",239000.0,apartment
2,Apartment In Puerto De Andratx,"Renovated sea view studio with kitchenette and bathroom with shower. Private terrace with dining table, chairs and sunloungers. Excellent sea views and access to Port Andratx facilities",170000.0,apartment


In [107]:
recommender('cheap apartments by the beach')

,Listing Name,Description,Price,Type
0,Mallorca Apartments Apartment For Sale In Santa Ponsa Very Close To All Amenities,,150000.0,apartment
1,Down Floor Apartment In Talamanca,,350000.0,apartment
2,Nice Apartment For Sale In Puerto Pollensa,"This apartment is located in the popular area of Puerto Pollensa Pinaret. It is a ground floor apartment with a beautiful terrace, the perfect place to relax. The apartment has two bedrooms and a bathroom, Kücheund a fairly large dining room. It is located just 50 meters from the beach and in general there are good shopping. The residential area of Puerto Pollensa Pinaret is one of the most exclusive areas of the city because of its proximity to the pedestrian promenade, restaurants, shops and beach.",215000.0,apartment
3,Bedroom Apartment Overlooking The Little Beach In Santa Ponsa With Sea Views And Parking Place,"2 bedroom apartment overlooking the little beach in Santa Ponsa. This bright apartment is within a 4 minute walk to al amenities Santa Ponsa has to offer such as shops, restaurants and the big beach. There is also a parking space sold with this property.",275000.0,apartment
4,Mallorca Property Apartment For Sale In Santa Ponsa Close To All Amenities,,90000.0,apartment


In [94]:
import streamlit as st